# 5. CML API を使って Chroma ベクターDBを作成するジョブを定義する

演習2では、依存ジョブの作成を手動で行いました。 この演習では、同じことを CML APIv2 を使って行います。  
CML API を使用する利点は、ジョブの作成・実行のためのプログラム的なアプローチをとれることです。
ジョブを作成するためにcmlapiライブラリを使用すると、自動化、バージョン管理、再現性、統合、スケーラビリティ、エラー処理、ジョブ管理を効率化でき、データ処理のワークフローを合理化するという点で有益です。

![Populate Chroma architecture](../assets/exercise_5.png)

#### 5.1 インポートの宣言、CML API クライアントの作成、利用可能なランタイムの一覧表示

- 必要なモジュールのインポート
- コレクション名の定義
- CML クライアントの初期化、特定の 条件に合致する利用可能なランタイムのリストの取得、利用可能なランタイムの一覧表示

In [1]:
import os
import cmlapi
import random
import string
import json

COLLECTION_NAME = 'cml-default' ## Update if you have changed this
    
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Nvidia GPU",
    "editor": "JupyterLab"
}))

#### 5.2 最新のMLランタイム識別子を取得し、環境変数に保存する

In [2]:
## 利用可能なランタイムを環境の最新のランタイムに設定する (イテレータは0から始まり、順次進む番号です)
## JOB_IMAGE_ML_RUNTIME 変数には、ジョブの起動に使われるMLランタイムを格納します。
print(available_runtimes.runtimes[0])
print(available_runtimes.runtimes[0].image_identifier)
JOB_IMAGE_ML_RUNTIME = available_runtimes.runtimes[0].image_identifier

## ランタイムを環境変数に保存します
## こうすることで、何度もランタイムを取得しなおす必要がなくなります
os.environ['JOB_IMAGE_ML_RUNTIME'] = JOB_IMAGE_ML_RUNTIME

{'description': 'Python runtime with CUDA libraries provided by Cloudera',
 'edition': 'Nvidia GPU',
 'editor': 'JupyterLab',
 'full_version': '2024.02.1-b4',
 'image_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.02.1-b4',
 'kernel': 'Python 3.10',
 'register_user_id': 0,
 'status': 'ENABLED'}
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.02.1-b4


#### 5.3 現在のプロジェクトを取得
現在のプロジェクトのメタデータを取得し表示します

In [3]:
# 現在のプロジェクトの識別子を表示
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))

#### 5.4 Chroma DB を作成するジョブの作成と実行

このコードは、ランダムな識別子を生成し、プロジェクトID、スクリプト、リソース割り当てなどの指定されたパラメータを使用して、Chroma Vectorデータベースに入力るためのジョブリクエストを作成します。  
その後、Cloudera Machine Learning環境内でジョブと対応するジョブの実行を作成し、ベクトルDBにデータを入力するタスクを開始します。

In [4]:
random_id=''.join(random.choice(string.ascii_lowercase) for i in range(10))
job_body = cmlapi.CreateJobRequest(
    project_id = project.id,
    name = "Populate Chroma Vector DB " + random_id, 
    script = "5_populate_local_chroma_db/populate_chroma_vectors.py",
    cpu = 1,
    memory = 4,
    runtime_identifier = os.getenv('JOB_IMAGE_ML_RUNTIME')
)

job_result = client.create_job(
    body = job_body, 
    project_id = str(project.id)
)

job_run = client.create_job_run(
    cmlapi.CreateJobRunRequest(),
    project_id = project.id, 
    job_id = job_result.id
)